In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None

In [2]:
with open('match.json') as file:
    data = json.load(file)

In [3]:
data

{'meta': [],
 'events': [{'id': 1074868841,
   'matchId': 5234875,
   'matchPeriod': '1H',
   'minute': 0,
   'second': 14,
   'matchTimestamp': '00:00:14.897',
   'videoTimestamp': '20.897823',
   'relatedEventId': 1074868842,
   'type': {'primary': 'pass',
    'secondary': ['back_pass', 'short_or_medium_pass']},
   'location': {'x': 49, 'y': 50},
   'team': {'id': 1612, 'name': 'Liverpool', 'formation': '4-3-3'},
   'opponentTeam': {'id': 1625,
    'name': 'Manchester City',
    'formation': '4-3-3'},
   'player': {'id': 7964, 'name': 'J. Henderson', 'position': 'RCMF3'},
   'pass': {'accurate': True,
    'angle': 149,
    'height': None,
    'length': 29,
    'recipient': {'id': 14870, 'name': 'J. Matip', 'position': 'RCB'},
    'endLocation': {'x': 25, 'y': 72}},
   'shot': None,
   'groundDuel': None,
   'aerialDuel': None,
   'infraction': None,
   'carry': None,
   'possession': {'id': 1074868841,
    'duration': '21.876562',
    'types': [],
    'eventsNumber': 10,
    'eventIn